In [1]:
#import the librbary
import psycopg2

In [2]:
!pip install psycopg2

In [3]:
try:
    conn = psycopg2.connect(host='localhost',dbname="Upflowy_project_db",user="postgres",password='Pa55word')
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
    
conn.set_session(autocommit=True)

In [4]:
# create a cursor
cur = conn.cursor()

In [4]:
#create database
cur.execute("CREATE DATABASE flight;")

DuplicateDatabase: database "flight" already exists


create spark session object and import postgresql connector jar file in current spark session object

In [5]:
#import pyspark library
!pip install pyspark

In [ ]:
#import os
#from pyspark.sql import SparkSession

#PATH_TO_JAR_FILE = "postgresql-42.3.4.jar"
#spark = SparkSession \
    #.builder \
    #.appName("PySpark_POSTGRESQL") \
    #.config("spark.jars", PATH_TO_JAR_FILE) \
    #.getOrCreate()

#DB_HOST = os.environ.get("PG_HOST")
#DB_PORT = os.environ.get("PG_PORT")
#DB_NAME = os.environ.get("PG_DB_CLEAN")
#DB_PASSWORD = os.environ.get("PG_PASSWORD")
#DB_USER = os.environ.get("PG_USERNAME")

#df = spark.read \
   # .format("jdbc") \
   # .option("url", f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}") \
    #.option("user", DB_USER) \
    #.option("password", DB_PASSWORD) \
    #.option("driver", "org.postgresql.Driver") \
    #.option("query","select * from your_table") \
    #.option('fetchsize',"1000") \
    #.load()

#df.printSchema()

In [7]:
spark.stop()

In [9]:
#import spark session
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.jars", "postgresql-42.3.4.jar") \
    .master("local").appName("PySpark_POSTGRESQL").getOrCreate()

In [10]:
#create pyspark dataframe
AirlineDF = spark.read.option("header", "true").csv("airlines1.csv")

In [11]:
AirlineDF.show(5)

22/05/11 23:16:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+--------------------+---------+-------------+--------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----

22/05/11 23:16:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLan

In [13]:
AirlineDF5 = AirlineDF.select("_c0","Year","Quarter", "Month", "DayofMonth", "DayofWeek" , \
                              "FlightDate","Reporting_Airline","DOT_ID_Reporting_Airline", \
                              "OriginCityName" , "OriginState", "OriginStateFips", "OriginStateName", \
                              "Distance" , "DistanceGroup","CarrierDelay","WeatherDelay", "NASDelay")

In [14]:
AirlineDF5.show(5)

+---+----+-------+-----+----------+---------+----------+-----------------+------------------------+------------------+-----------+---------------+---------------+--------+-------------+------------+------------+--------+
|_c0|Year|Quarter|Month|DayofMonth|DayofWeek|FlightDate|Reporting_Airline|DOT_ID_Reporting_Airline|    OriginCityName|OriginState|OriginStateFips|OriginStateName|Distance|DistanceGroup|CarrierDelay|WeatherDelay|NASDelay|
+---+----+-------+-----+----------+---------+----------+-----------------+------------------------+------------------+-----------+---------------+---------------+--------+-------------+------------+------------+--------+
|  0|1998|      1|    1|         2|        5|1998-01-02|               NW|                   19386|   Minneapolis, MN|         MN|           27.0|      Minnesota|   991.0|            4|        null|        null|    null|
|  1|2009|      2|    5|        28|        4|2009-05-28|               FL|                   20437|     Milwaukee, W

22/05/11 23:19:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, OriginCityName, OriginState, OriginStateFips, OriginStateName, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay
 Schema: _c0, Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, OriginCityName, OriginState, OriginStateFips, OriginStateName, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay
Expected: _c0 but found: 
CSV file: file:///Users/denny/pysparktest/code/PySpark%20ETL%20Project-Build%20a%20Data%20Pipeline%20using%20S3%20and%20MySQL/airlines1.csv


Write data from pyspark dataframe to Mysql Database

In [15]:
AirlineDF5.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5432/flight',
      driver='org.postgresql.Driver',
      dbtable='Airline',
      user='postgres',
      password='Pa55word').mode('append').save()



22/05/11 23:19:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, OriginCityName, OriginState, OriginStateFips, OriginStateName, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay
 Schema: _c0, Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, OriginCityName, OriginState, OriginStateFips, OriginStateName, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay
Expected: _c0 but found: 
CSV file: file:///Users/denny/pysparktest/code/PySpark%20ETL%20Project-Build%20a%20Data%20Pipeline%20using%20S3%20and%20MySQL/airlines1.csv


In [ ]:
CREATE TABLE employees (name STRING, dept STRING, salary INT, age INT);

INSERT INTO employees VALUES ("Lisa", "Sales", 10000, 35);
INSERT INTO employees VALUES ("Evan", "Sales", 32000, 38);
INSERT INTO employees VALUES ("Fred", "Engineering", 21000, 28);
INSERT INTO employees VALUES ("Alex", "Sales", 30000, 33);
INSERT INTO employees VALUES ("Tom", "Engineering", 23000, 33);
INSERT INTO employees VALUES ("Jane", "Marketing", 29000, 28);
INSERT INTO employees VALUES ("Jeff", "Marketing", 35000, 38);
INSERT INTO employees VALUES ("Paul", "Engineering", 29000, 23);
INSERT INTO employees VALUES ("Chloe", "Engineering", 23000, 25);


In [13]:
spark.stop()

In [ ]:
try:
    cur.execute("""SELECT
    productname,
    productid,
    round(( AVG( is_click )*100), 2) CTR
FROM
    product p
INNER JOIN ad_click_prediction_train ad
        USING(productid)
GROUP BY
    productname, productid
ORDER BY
    CTR DESC LIMIT 5
    """)
    CTR_records = cur.fetchall()
    print("The top 5 product names with highest click through rate.")
    for row in CTR_records:
        print("ProductName = ", row[0], )
        print("Product_Id = ", row[1])
        print("CTR  = ", row[2], "\n")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)